In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from bs4 import BeautifulSoup
import requests

In [2]:
CSV_files ={"customers": r"C:\Users\mzanola\project\.venv\final_project\dataset\olist_customers_dataset.csv",
"geolocation_dataset":r"C:\Users\mzanola\project\.venv\final_project\dataset\olist_geolocation_dataset.csv",
"order_items":r"C:\Users\mzanola\project\.venv\final_project\dataset\olist_order_items_dataset.csv",
"order_payments":r"C:\Users\mzanola\project\.venv\final_project\dataset\olist_order_payments_dataset.csv",
"order_review":r"C:\Users\mzanola\project\.venv\final_project\dataset\olist_order_reviews_dataset.csv",
"order_dataset":r"C:\Users\mzanola\project\.venv\final_project\dataset\olist_orders_dataset.csv",
"list_product":r"C:\Users\mzanola\project\.venv\final_project\dataset\olist_products_dataset.csv",
"list_seller":r"C:\Users\mzanola\project\.venv\final_project\dataset\olist_sellers_dataset.csv",
"product_category":r"C:\Users\mzanola\project\.venv\final_project\dataset\product_category_name_translation.csv"}

In [3]:
dataframes = {name: pd.read_csv(path) for name, path in CSV_files.items()}

In [4]:
df_orders_items = dataframes["order_items"]
df_order_payments = dataframes["order_payments"]
df_order_review= dataframes["order_review"]
df_order_dataset = dataframes["order_dataset"]
df_list_product = dataframes["list_product"]
df_list_seller = dataframes["list_seller"]
df_product_category = dataframes["product_category"]
df_geolocation_dataset = dataframes["geolocation_dataset"]

In [40]:
df_list_product.info()#individuati 610 valori nulli sul df list_:product

<class 'pandas.DataFrame'>
RangeIndex: 32951 entries, 0 to 32950
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   product_id                  32951 non-null  str    
 1   product_category_name       32341 non-null  str    
 2   product_name_lenght         32341 non-null  float64
 3   product_description_lenght  32341 non-null  float64
 4   product_photos_qty          32341 non-null  float64
 5   product_weight_g            32949 non-null  float64
 6   product_length_cm           32949 non-null  float64
 7   product_height_cm           32949 non-null  float64
 8   product_width_cm            32949 non-null  float64
dtypes: float64(7), str(2)
memory usage: 2.3 MB


In [ ]:
df_list_product

In [ ]:
product_id_nan=df_list_product[df_list_product['product_category_name'].isna()]
product_id_to_delete=product_id_nan['product_id']
product_id_to_delete=product_id_to_delete.to_numpy()
#creato un array con gli id prodototto che non hanno categorie per usarlo come criteri di esclusione negli altri df

In [60]:
df_list_product=df_list_product.dropna()
df_list_product.info()#pulito il df  da i valori nulli probabilemnte ho perso 1 id product 32340 invece di  32341

<class 'pandas.DataFrame'>
Index: 32340 entries, 0 to 32950
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   product_id                  32340 non-null  str    
 1   product_category_name       32340 non-null  str    
 2   product_name_lenght         32340 non-null  float64
 3   product_description_lenght  32340 non-null  float64
 4   product_photos_qty          32340 non-null  float64
 5   product_weight_g            32340 non-null  float64
 6   product_length_cm           32340 non-null  float64
 7   product_height_cm           32340 non-null  float64
 8   product_width_cm            32340 non-null  float64
dtypes: float64(7), str(2)
memory usage: 2.5 MB


In [61]:
df_orders_items_clean=df_orders_items[~df_orders_items['product_id'].isin(product_id_to_delete)]
df_orders_items_clean.info()#creato il dataframe df_orders ripulito dai prodotti senza categoria

<class 'pandas.DataFrame'>
Index: 111047 entries, 0 to 112649
Data columns (total 7 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   order_id             111047 non-null  str    
 1   order_item_id        111047 non-null  int64  
 2   product_id           111047 non-null  str    
 3   seller_id            111047 non-null  str    
 4   shipping_limit_date  111047 non-null  str    
 5   price                111047 non-null  float64
 6   freight_value        111047 non-null  float64
dtypes: float64(2), int64(1), str(4)
memory usage: 6.8 MB
